![http://cognitiveatlas.org//static/img/logo-front.png](http://cognitiveatlas.org//static/img/logo-front.png)


<hr>

Here we are going to interact with the [Cognitive Atlas](https://www.cognitiveatlas.org) 
via it's [Python API](https://github.com/CognitiveAtlas/cogat-python).

## What is the Cognitive Atlas?

The Cognitive Atlas is a community developed knowledge base for concepts defined in cognitive psychology. More formally, this is called an ontology, and less formally, it's a place to look up what a term means that you see in the psychology literature, and how it relates to other terms.

We start using the cognitive atlas when we have a question of interest.

> What is working memory?

That seems like a good question! How might we have investigated the question, without any additional tool?

 - Do a Google Search and trust the internet
 - Do a literature review and try to synthesize many results
 - Rely on meta analysis
 
The above options are good, but each has drawbacks. For example, it would be hard to synthesize the opinions and studies from any of the resources above. Meta analyses are good, 

<hr>

## The Web Interface

If you are just exploring, you are best to interact with the Cognitive Atlas via it's web interface at [cognitiveatlas.org](https://www.cognitiveatlas.org). For example, here is a page for working memory at [http://cognitiveatlas.org/concept/id/trm_4a3fd79d0b5a7/](http://cognitiveatlas.org/concept/id/trm_4a3fd79d0b5a7/). I'm showing you the URL for a few important reasons:

  - we see that it's something called a "concept"
  - we see embedded in the URL is some kind of "id," the string `trm_4a3fd79d0b5a7`. Let's keep this in mind for later.

### Use Cases

From the interface we can quickly learn many things like the following:

### 1. Definition

We might be writing a review or paper, or just want to understand what the term means, to the best of the knowledge of the experts that have contributed to the Cognitive Atlas. Here we can see, front and center, this definition (here is the first bit of it).

> "active maintenance and flexible updating of goal/task relevant information (items, goals, strategies, etc.) in a form that resists interference but has limited capacity. ..."

This is great, but note that it hasn't been reviewed. We can do more investigation by looking at the [citations](http://cognitiveatlas.org/concept/id/trm_4a3fd79d0b5a7/#block-biblio) on the page, or in the case of a non Anonymous contribution, see the contribution author(s). Note that although you can't see it, there is a complete history kept of contributors to the terms.

### 2. Tasks

If you are a graduate student and wanting to conduct a study to measure working memory, this might be your stopping point! There is a list of over 15 [tasks](http://cognitiveatlas.org/concept/id/trm_4a3fd79d0b5a7/#block-measurements) on the page that have been asserted to measure working memory.

### 3. Relationships

How does working memory relate to the concept of [maintenance](http://cognitiveatlas.org/concept/id/trm_4a3fd79d0ba30/)? What about general [decision making](http://cognitiveatlas.org/concept/id/trm_4a3fd79d0a038/)? Remember that the Cognitive Atlas is an ontology, which means it defines both concepts **and** relationships between them! We have descriptors for these relationships, like "part of" or "kind of". You might see how this maps nicely to a graph:

 - a **concept** or **task** (or general entity represented in the atlas) maps to a node
 - a **relationship** between terms maps to a link between nodes
 - the strength of any relationship maps to a weight of a link between nodes
 
 This opens up a very powerful idea - we can use ideas and methods from graph theory to understand ideas in cognitive science. We can also generate visualizations to see them. For example, try posting this query:
 
 ```cypher
 MATCH (n:concept) where n.id = 'trm_4a3fd79d0ba30' OPTIONAL MATCH (n)-[]-(r) return n, r
 ```
Into the Neo4J browser (the top line of the window at [https://cognitiveatlas.org:7473/browser/](https://cognitiveatlas.org:7473/browser/)). You would see a little graph that describes the term "maintenance"

![neo4j-graph-maintenance.png](neo4j-graph-maintenance.png)

While many of these interactive, jumpy visualizations aren't super useful, they can be helpful for a talk, to show an idea in a paper, or solidify your understanding of a more complex relationship.

These are the basic functions and components of the Cognitive Atlas. If you are interested, you can also explore [Phenotypes](http://cognitiveatlas.org/disorders), [Tasks](http://cognitiveatlas.org/tasks/a/) and [Collections](http://cognitiveatlas.org/collections) (groups of tasks). But the web interface is limited in what we are able to analyze, in a large scale sense. In the next section, let's talk about more programmatic use of the Cognitive Atlas.

<hr>

## The Cognitive Atlas API

API means "application programming interface" and it really is exactly that! It's a set of functions that you can call from a programming language (e.g., Python and R are common languages for scientific programming) to interact with a resource (for example, a database) to answer some question. In fact, APIs are essential for science - there are questions that, in order to answer, might require interaction with datasets beyond the ability of a human. We can't easily answer all queries that we might have about cognitive knowledge in a web browser! For example, here are some more advanced use cases that would require programming and (some) graph theory:

 - What cognitive concepts are represented across a database of over 1 million articles from the literature
 - **how** similar is working memory to all other concepts in the atlas?
     - Can you give me a number?
     - Can you define a threshold of significance?
 - What parts of the brain, represened by coordinate locations reported in the literature, are associated with the concept?

Beyond using the Cognitive Atlas as a lookup for terms, you can see that there are many use cases in research that would warrant interacting with it programatically, integrated with other tools, or scaled. Today let's explore one of these use cases - the third - and understand what areas of the brain might be involved with working memory.
 
<hr>
 
# Getting Started Tutorial

The reason we are using a Jupyter Notebook is so that we can have code! If you are curious, this was generated with a Python 3 kernel, and installing the [Cognitive Atlas](https://github.com/CognitiveAtlas/cogat-python) Python API wrapper with pip:

```bash
pip install cognitiveatlas
```

Actually, the API itself is a [RESTful](https://codewords.recurse.com/issues/five/what-restful-actually-means) API, and could be interacted with via any programming language that can parse the [JSON endpoints](https://cognitiveatlas.org/api). Python is a popular language for neuroscience, and so it was chosen by the creator, who happens to be [this author](https://www.github.com/vsoch) :) It's a small world!

## 1. Define interesting questions

We don't want to approach a programming task without having clear definition of what we are aiming to do. Let's define a simple set of questions right now:

 1. What is working memory?
 2. What concepts are related to it?
 3. What parts of the brain have associations from the literature?
 
If we were using the interface we might have been okay going to the page discussed earlier, but we run into some trouble with the third question, because we need to integrate with another database (called [NeuroSynth](http://neurosynth.org) to derive a brain map, more on this later) and, while there actually is a button to drive this search now in the browser, we might want to easily do many of these queries, and for each, to download result maps at scale. That would be a lot of clicking to do in the interface! With programming, we can write a script and then go off and make a sandwich. Okay, so we have our dummy question of interest. 


## 2. Define plan of action

Once we have questions of interest, we need a (general) sense of how we will approach solving the problem. For the above, we are going to take the following approach:

 1. Use the Cognitive Atlas Python API to query the Cognitive Atlas to look up a term based on it's name
 2. From 1., use the term identifier, and use that identifier to look for terms that are related.
 3. Use the same identifier to query the NeuroSynth API to derive a brain map.
 
I'll leave reading about how [NeuroSynth works](http://neurosynth.org/faq/) to the interested reader.

Let's get started!

<hr>

### Look Up a Term

Let's import functions that we need from the Cognitive Atlas API, namely those to "get" things.

In [47]:
from cognitiveatlas.api import get_concept, get_task, get_disorder

How would you know how to use these functions? if you follow the link from the Cognitive Atlas site
you are taken to the [readthedocs](https://cogat-python.readthedocs.io/en/latest/getting_started.html#get-concept) pages. ReadTheDocs is a service that can help software maintainers to easily render and provide beautiful
documentation for their users. This is where we will find examples, and how to provide arguments to a function.
This is important for our first interaction, and a general good piece of advice.

> It's helpful to start learning with a tutorial or "getting started" guide

If a piece of software _doesn't_ provide this, you should ask for it. Whether that is a post on an
issue board or an emali to a list, as a user it's an important resource to be provided for you.
And it's ok to ask for what you need. Okay, so based on the page linked above for the function `get_concept`
let's see if we get get the concept "working memory":

In [19]:
concept = get_concept(name="working memory")

http://cognitiveatlas.org/api/v-alpha/concept?name=working%20memory
Result Includes:<pandas:data frame><json:dict><txt:str><url:str>


Hey that's pretty neat! The call is showing me a quick link to see the concept, along with how to interact with it. It looks like the result has a [pandas](https://pandas.pydata.org/) data frame, a [JSON](http://json-schema.org/) (JavaScript Object Notation) object, and the url as a string. Let's see what we get when we print it: Note that if you didn't know what was in the result (and the documentation didn't help you) you could type `dir(concept)` to see it, or in an interactive jupyter notebook or ipython terminal you could use TAB autocompletion.

In [25]:
# Print the keys for the dictionary
print("The keys for the concept %s are:" % concept.json['name'])
print(concept.json.keys())

The keys for the concept working memory are:
dict_keys(['conceptclasses', 'definition_text', 'contrasts', 'def_id', 'event_stamp', 'id_concept_class', 'id_user', 'def_event_stamp', 'id', 'def_id_user', 'type', 'citations', 'alias', 'last_updated', 'creation_time', 'concepts', 'relationships', 'name'])


And then we can look at the entire thing, if we wish, which includes the values for those keys!

In [26]:
concept.json

{'alias': '',
 'citations': [{'citation_authors': 'Cowan N',
   'citation_comment': 'Via PubMed API',
   'citation_desc': 'The magical number 4 in short-term memory: a reconsideration of mental storage capacity.',
   'citation_pmid': '11515286',
   'citation_pubdate': '2001 Feb',
   'citation_pubname': ' (Behav Brain Sci)',
   'citation_source': 'PubMed API',
   'citation_type': 'CT2',
   'citation_url': 'http://www.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi?dbfrom=pubmed&id=11515286&retmode=ref&cmd=prlinks',
   'event_stamp': '2012-06-25T22:30:39',
   'id': 'cit_4fe8e68f71ca2',
   'id_user': 'usr_4f177506dde77',
   'relationship': 'HASCITATION'},
  {'citation_authors': 'Baddeley A',
   'citation_comment': 'Via PubMed API',
   'citation_desc': 'Working memory.',
   'citation_pmid': '1736359',
   'citation_pubdate': '1992 Jan 31',
   'citation_pubname': ' (Science)',
   'citation_source': 'PubMed API',
   'citation_type': 'CT2',
   'citation_url': 'http://www.ncbi.nlm.nih.gov/entrez/euti

Wow, that's quite a bit of metadata and information about "working memory" that we just didn't see in the web interface. This also tends to be the case for APIs - they provide a richer source of information. We can easily see the definition of working memory under the "definition" key:

In [27]:
print(concept.json['definition_text'])

active maintenance and flexible updating of goal/task relevant information (items, goals, strategies, etc.) in a form that resists interference but has limited capacity. These representations may involve flexible binding of representations, may be characterized by the absence of external support for the internally maintained representations, and are frequently temporary due to ongoing interference



Boum! Does that look familiar? We just found the exact definition that we read in the web interface earlier! It's even extracted from the same database, likely with the same function on the back end. Now imagine if you had a list of terms that you would otherwise have to manually look up? You can write a loop to do it for you!

In [46]:
for term in ['anxiety', 'memory', 'negative emotion']:
    concept = get_concept(name=term)

http://cognitiveatlas.org/api/v-alpha/concept?name=anxiety
Result Includes:<pandas:data frame><json:dict><txt:str><url:str>
http://cognitiveatlas.org/api/v-alpha/concept?name=memory
Result Includes:<pandas:data frame><json:dict><txt:str><url:str>
http://cognitiveatlas.org/api/v-alpha/concept?name=negative%20emotion
Result Includes:<pandas:data frame><json:dict><txt:str><url:str>


Note that we've even found a term that isn't defined yet, "negative emotion." If I were interested or working on this term, I would want to open up discussion with others to define this term. We can also use the API to look up concepts based on other fields, or the same lookup for tasks or disorders:

In [45]:
task = get_task(name="Stroop task")

http://cognitiveatlas.org/api/v-alpha/task?name=Stroop%20task
Result Includes:<pandas:data frame><json:dict><txt:str><url:str>


In [44]:
phenotype = get_disorder(name="major depressive disorder")

http://cognitiveatlas.org/api/v-alpha/disorder?name=major%20depressive%20disorder
Result Includes:<pandas:data frame><json:dict><txt:str><url:str>


I think at this point we've successfully used the API to answer our first question! You also have a sense of how this could scale. Let's move on to the second.

<hr>

### Look for related terms

Our next task is to look for related terms. Let's again get our concept, and switch it up a bit and look at "anxiety" instead (one that I was particularly interested in during graduate school!) We will get the concept, and then look at the relationships to it. This would be akin to manually walking down a link on the graph to find another node.

In [49]:
concept = get_concept(name="anxiety")
concept.json['relationships']

http://cognitiveatlas.org/api/v-alpha/concept?name=anxiety
Result Includes:<pandas:data frame><json:dict><txt:str><url:str>


[{'alias': '',
  'creation_time': 1512660629809,
  'def_event_stamp': '2012-03-28T17:54:40',
  'def_id': 'def_4f7350605133e',
  'def_id_user': 'usr_49a467bf4e0db',
  'definition_text': 'a complex of psychological phenomena that involve some degree of arousal and valence (positive/negative)',
  'direction': 'parent',
  'event_stamp': '2009-06-22T19:12:55',
  'id': 'trm_4a3fd79d0a17f',
  'id_concept_class': 'ctp_C8',
  'id_user': 'usr_0000000000',
  'last_updated': 1512660629809,
  'name': 'emotion',
  'relationship': 'KINDOF'}]

Above we learn that anxiety is a `KIND OF` emotion. We could then look up this term (the walking along the graph part):

In [50]:
parent = get_concept(concept.json['relationships'][0]['id'])

http://cognitiveatlas.org/api/v-alpha/concept?id=trm_4a3fd79d0a17f
Result Includes:<pandas:data frame><json:dict><txt:str><url:str>


<hr>

### Derive a Brain Map

Here is where it gets fun! if we use the term identifier and a different API, the one from NeuroSynth, we can derive a complete brain map for our term. How? We are going to do something that looks like this:

**TODO** Why would I want to use the API here (what is a good example?) I could just get a link in the interface,
then go to the url with the brainmaps to download: http://neurosynth.org/analyses/terms/anxiety/

You can think of NeuroSynth as a framework to do a meta analysis that looks across the psychology literature, finds tables of x,y,z coordinates reported in publications, and looks for "significant" areas of agreement. Aka, what blobs in the brain are commonly seen in papers with a high prevalence of the term "anxiety" in the abstract? It's a fairly simple method, but it (surprising to the author as well, I think) works fairly well. Since NeuroSynth doesn't have a Python API wrapper, we will interact with the endpoint directly using the [requests](http://docs.python-requests.org/en/master/) library.

**TODO** Derive steps here, the API functions are [documented here](http://neurosynth.org/api/v2/). It's not clear to me what a "common use case" looks like.